In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [8]:
df=pd.read_csv('/content/Phishing_site.csv')

In [9]:
df.shape

(127723, 3)

In [10]:
X = df['URL'].astype(str)
y = df['label']

In [11]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_seq, maxlen=100)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [27]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
1278/1278 [==============================] - 52s 35ms/step - loss: 0.1248 - accuracy: 0.9491 - val_loss: 0.0813 - val_accuracy: 0.9686
Epoch 2/20
1278/1278 [==============================] - 27s 21ms/step - loss: 0.0676 - accuracy: 0.9739 - val_loss: 0.0714 - val_accuracy: 0.9725
Epoch 3/20
1278/1278 [==============================] - 29s 22ms/step - loss: 0.0539 - accuracy: 0.9784 - val_loss: 0.0696 - val_accuracy: 0.9720
Epoch 4/20
1278/1278 [==============================] - 30s 24ms/step - loss: 0.0456 - accuracy: 0.9823 - val_loss: 0.0708 - val_accuracy: 0.9727
Epoch 5/20
1278/1278 [==============================] - 27s 21ms/step - loss: 0.0406 - accuracy: 0.9837 - val_loss: 0.0784 - val_accuracy: 0.9732


In [43]:
from keras.preprocessing.sequence import pad_sequences

def predict_url_phishing(url, tokenizer, model):
    # Preprocess the URL
    url_seq = tokenizer.texts_to_sequences([url])
    url_padded = pad_sequences(url_seq, maxlen=100)
    prediction = model.predict(url_padded)
    if prediction > 0.5:
        return "Phishing Site"
    else:
        return "Not Phishing Site"

url_to_predict = "www.financialaccounting.com/financialprof.htm"
prediction = predict_url_phishing(url_to_predict, tokenizer, model)
print(prediction)



1/1 [==============================] - 0s 24ms/step
Phishing Site


In [36]:
filtered_df = df[df['label'] == 1]
phishing_urls = filtered_df['URL']
print(phishing_urls)


18231    esxcc.com/js/index.htm?us.battle.net/noghn/en/...
18232    wwweira¯&nvinip¿ncH¯wVö%ÆåyDaHðû/ÏyEùuË\nÓ6...
18233    'www.institutocgr.coo/web/media/syqvem/dk-óij...
18234                    YìêkoãÕ»Î§DéÎl½ñ¡ââqtò¸/à; Í
18236    ruta89fm.com/images/AS@Vies/1i75cf7b16vc<Fd16...
                               ...                        
96000              xbox360.ign.com/objects/850/850402.html
96001         games.teamxbox.com/xbox-360/1860/Dead-Space/
96002           www.gamespot.com/xbox360/action/deadspace/
96003        en.wikipedia.org/wiki/Dead_Space_(video_game)
96004            www.angelfire.com/goth/devilmaycrytonite/
Name: URL, Length: 48103, dtype: object


In [37]:
phishing_urls=list(phishing_urls)

In [42]:
phishing_urls[99]

'www.financialaccounting.com/financialprof.htm'

In [44]:
loss, accuracy = model.evaluate(X_test, y_test)

799/799 [==============================] - 6s 8ms/step - loss: 0.0775 - accuracy: 0.9701


In [45]:
model.save('/content/Phising_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
